In [1]:
import sys
sys.path.append('C:/ROOT_final/final_project')
from pling_evaluate import write_json, generate_result, ArenaEvaluator

In [2]:
import pickle
import pandas as pd
import os
import io
import json
import distutils.dir_util
import numpy as np
from pling_evaluate import write_json, generate_result, ArenaEvaluator

# AutoEncoder Basic

> Data Load

In [4]:
with open('pickle/train_onehot.pkl', 'rb') as f:
    train_onehot = pickle.load(f)

train_onehot.shape

(37406, 91697)

In [3]:
with open('pickle/test_q_onehot.pkl', 'rb') as f:
    test_q_onehot = pickle.load(f)

test_q_onehot.shape

(9352, 91697)

In [4]:
with open('pickle/mfl_col.pkl', 'rb') as f:
    mfl_col = pickle.load(f)

len(mfl_col)

91697

In [5]:
song_len = 88146
# song = 88146, tag = 3551

In [6]:
test_q = pd.read_json('data/test_cnt3_q.json')
test_q.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9352 entries, 0 to 9351
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          9352 non-null   object
 1   id            9352 non-null   int64 
 2   plylst_title  9352 non-null   object
 3   songs         9352 non-null   object
 4   like_cnt      9352 non-null   int64 
 5   updt_date     9352 non-null   object
dtypes: int64(2), object(4)
memory usage: 438.5+ KB


> Modeling

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import Adam

> model_7
- input_dim = 91697 #len(mfl_col)
- hidden_dim = 450
- dropout = 0.2
- BatchNormalization(input_shape=(hidden_dim,))
- LeakyReLU()
- epoch = 20
- batch_size = 256
- loss_function = binary_crossentropy
- learning_rate = 0.0005
- optimizer = adam(learning_rate=learning_rate)

---

In [44]:
input_dim = 91697 #len(mfl_col)
hidden_dim = 450
dropout_rate = 0.2

inputs = Input(shape=(input_dim,))
encoded = Dropout(0.2)(inputs)
encoded = Dense(hidden_dim)(encoded)
encoded = BatchNormalization(input_shape=(hidden_dim,))(encoded)
encoded = LeakyReLU()(encoded)

decoded = Dense(input_dim,activation='sigmoid')(encoded)
model = Model(inputs, decoded, name='autoencoder')
model.summary()

Model: "autoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 91697)]           0         
                                                                 
 dropout (Dropout)           (None, 91697)             0         
                                                                 
 dense (Dense)               (None, 450)               41264100  
                                                                 
 batch_normalization (Batch  (None, 450)               1800      
 Normalization)                                                  
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 450)               0         
                                                                 
 dense_1 (Dense)             (None, 91697)             41355347  
                                                       

In [8]:
model = tf.keras.models.load_model('results/model_ad_e30.h5')

In [7]:
learning_rate = 0.0005
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer,loss='binary_crossentropy')

In [8]:
model.fit(train_onehot, train_onehot, epochs=5, batch_size=256, validation_split=0.2, verbose=1)

Epoch 1/5
117/117 [==============================] - 379s 3s/step - loss: 0.0019 - val_loss: 0.0022
Epoch 2/5
117/117 [==============================] - 345s 3s/step - loss: 0.0019 - val_loss: 0.0022
Epoch 3/5
117/117 [==============================] - 345s 3s/step - loss: 0.0019 - val_loss: 0.0022
Epoch 4/5
117/117 [==============================] - 348s 3s/step - loss: 0.0018 - val_loss: 0.0022
Epoch 5/5
117/117 [==============================] - 339s 3s/step - loss: 0.0018 - val_loss: 0.0022


In [9]:
model.save('results/model_ad_e30.h5')

C:\Users\SV\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [9]:
predict_plist=model.predict(test_q_onehot)

293/293 [==============================] - 21s 70ms/step


In [10]:
result = generate_result(test_q, mfl_col, song_len, predict_plist)

In [11]:
write_json(result,'results/results_ad_e30.json')

---

# 평가

In [12]:
gt_fname = 'data/test_cnt3_a.json'
rec_fname = 'results/results_ad_e30.json'
arena_evaluator = ArenaEvaluator()
arena_evaluator.evaluate(gt_fname, rec_fname)

Music nDCG: 0.137726
Tag nDCG: 0.419714
Score: 0.180024
